# Implementation of Beauregard's paper

### By Mathis Beaudoin (Summer 2024)

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


from shors_algorithm.methods.beauregard import cc_phi_add_mod, cc_phi_add_mod_dag, c_mult_mod, rev_mult_mod
from shors_algorithm.methods.utils.QFT import build_QFT, build_QFT_dag
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

## Control-control-add mod N

In [ ]:
c_integer = 16
q_integer = 4
N = 5

num_qubits = 8
register = [i for i in range(2, num_qubits - 1)]
ctrl_qubit_1 = 0
ctrl_qubit_2 = 1
ancillary = 7

add_mod = QuantumCircuit(num_qubits)
add_mod.x([ctrl_qubit_1, ctrl_qubit_2])
add_mod.initialize(f'{q_integer:b}'.zfill(num_qubits - 3), register)

add_mod.append(build_QFT(num_qubits - 3), register)
# cc_phi_add_mod_dag(add_mod, c_integer, N, ctrl_qubit_1, ctrl_qubit_2, ancillary, register) # Uncomment this for the inverse
cc_phi_add_mod(add_mod, c_integer, N, ctrl_qubit_1, ctrl_qubit_2, ancillary, register) # Comment this line for the inverse
add_mod.append(build_QFT_dag(num_qubits - 3), register)

result = Statevector(add_mod).probabilities_dict(decimals=10)
bit_string = list(result.keys())

print("cc_phi_add_mod N result :", result)
print("Integer result : ", int(bit_string[0][1:num_qubits-2],2))

# Multiplication mod N

In [ ]:
c_integer = 1
q_integer_1 = 1
q_integer_2 = 2
N = 5

num_qubits = 8
ctrl_qubit = 0 
register_1 = [i for i in range(1, num_qubits//2)] #result in this register
register_2 = [i for i in range(num_qubits//2, num_qubits - 1)] 
ancillary = num_qubits - 1

mult_mod = QuantumCircuit(num_qubits)
mult_mod.x(ctrl_qubit)
mult_mod.initialize(f'{q_integer_1:b}'.zfill(num_qubits//2 - 1), register_1)
mult_mod.initialize(f'{q_integer_2:b}'.zfill(num_qubits//2 - 1), register_2)

mult_mod.append(build_QFT(len(register_2)), register_2)
c_mult_mod(mult_mod, c_integer, N, ctrl_qubit, register_1, ancillary, register_2)
mult_mod.append(build_QFT_dag(len(register_2)), register_2)

result = Statevector(mult_mod).probabilities_dict(decimals=10)
bit_string = list(result.keys())

print("Mult mod result :", result)
print("Integer result : ", int(bit_string[0][1:num_qubits//2],2))

# Reversible multiplication mod N

In [ ]:
c_integer = 3
q_integer_1 = 3
q_integer_2 = 0
N = 7

num_qubits = 11
ctrl_qubit = 0
register_1 = [i for i in range(1, num_qubits - 6)]
register_2 = [i for i in range(num_qubits - 6, num_qubits - 1)]
ancillary = 10

s_u = QuantumCircuit(num_qubits)
s_u.x(0)
s_u.initialize(f'{q_integer_1:b}'.zfill(num_qubits - 7), register_1)
s_u.initialize(f'{q_integer_2:b}'.zfill(num_qubits - 6), register_2)

s_u.append(build_QFT(len(register_2)), register_2)
rev_mult_mod(s_u, c_integer, N, ctrl_qubit, register_1, ancillary, register_2)
s_u.append(build_QFT_dag(len(register_2)), register_2)

result = Statevector(s_u).probabilities_dict(decimals=10)
bit_string = list(result.keys())

print("rev_mult_mod result :", result)
print("Integer result : ", int(bit_string[0][num_qubits-6:num_qubits-1],2))